# California Housing Price Prediction
#### Author: Sayorn Chin
#### Date: 2021-07-19

### Objective

The project aims at building a model of housing prices to predict median house values in California using the provided dataset. This model should learn from the data and be able to predict the median housing price in any district, given all the other metrics.



### Import required libraries

In [2]:
import pandas as pd
import numpy as np

### Load dataset

In [4]:
## Assign the data frame as housing
housing = pd.read_excel('/Users/schinlfc/data-science-Python/california_housing_price_prediction/data/housing.xlsx')

In [5]:
## View the first 5 rows
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,median_house_value
0,-122.23,37.88,41,880,129.0,322,126,8.3252,NEAR BAY,452600
1,-122.22,37.86,21,7099,1106.0,2401,1138,8.3014,NEAR BAY,358500
2,-122.24,37.85,52,1467,190.0,496,177,7.2574,NEAR BAY,352100
3,-122.25,37.85,52,1274,235.0,558,219,5.6431,NEAR BAY,341300
4,-122.25,37.85,52,1627,280.0,565,259,3.8462,NEAR BAY,342200


In [6]:
## Get the number of rows and columns
housing.shape

(20640, 10)